In [2]:
library(tidyverse)
library(repr)
library(tidymodels)
library(rvest)
install.packages("GGally")
library("GGally")
options(repr.matrix.max.rows = 6)

Warning message:
“package ‘rvest’ was built under R version 4.0.2”
Loading required package: xml2


Attaching package: ‘rvest’


The following object is masked from ‘package:purrr’:

    pluck


The following object is masked from ‘package:readr’:

    guess_encoding


Warning message in install.packages("GGally"):
“installation of package ‘GGally’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2



In [4]:
shoppers_data_full <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00468/online_shoppers_intention.csv")
shoppers_data_full

Parsed with column specification:
cols(
  Administrative = col_double(),
  Administrative_Duration = col_double(),
  Informational = col_double(),
  Informational_Duration = col_double(),
  ProductRelated = col_double(),
  ProductRelated_Duration = col_double(),
  BounceRates = col_double(),
  ExitRates = col_double(),
  PageValues = col_double(),
  SpecialDay = col_double(),
  Month = col_character(),
  OperatingSystems = col_double(),
  Browser = col_double(),
  Region = col_double(),
  TrafficType = col_double(),
  VisitorType = col_character(),
  Weekend = col_logical(),
  Revenue = col_logical()
)



Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<lgl>
0,0,0,0,1,0,0.2,0.2,0,0,Feb,1,1,1,1,Returning_Visitor,FALSE,FALSE
0,0,0,0,2,64,0.0,0.1,0,0,Feb,2,2,1,2,Returning_Visitor,FALSE,FALSE
0,0,0,0,1,0,0.2,0.2,0,0,Feb,4,1,9,3,Returning_Visitor,FALSE,FALSE
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
0,0,0,0,6,184.25,0.08333333,0.08666667,0,0,Nov,3,2,1,13,Returning_Visitor,TRUE,FALSE
4,75,0,0,15,346.00,0.00000000,0.02105263,0,0,Nov,2,2,3,11,Returning_Visitor,FALSE,FALSE
0,0,0,0,3,21.25,0.00000000,0.06666667,0,0,Nov,3,2,1,2,New_Visitor,TRUE,FALSE


In [36]:
shoppers_data <- shoppers_data_full %>% 
    select(ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,Revenue) %>% 
    mutate(Revenue = as_factor(Revenue))
shoppers_split <- initial_split(data = shoppers_data, prop = 0.75, strata = Revenue)
shoppers_training <- training(shoppers_split) 
shoppers_testing <- testing(shoppers_split)
shoppers_training

ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,Revenue
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,0.000000,0.20,0.20,0,FALSE
1,0.000000,0.20,0.20,0,FALSE
2,2.666667,0.05,0.14,0,FALSE
⋮,⋮,⋮,⋮,⋮,⋮
5,465.75,0.00000000,0.02133333,0,FALSE
6,184.25,0.08333333,0.08666667,0,FALSE
3,21.25,0.00000000,0.06666667,0,FALSE


In [40]:
shoppers_recipe <- recipe(Revenue ~ ., data = shoppers_training) %>% 
    step_scale(all_predictors()) %>% 
    step_center(all_predictors())

In [41]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>% 
    set_engine("kknn") %>% 
    set_mode("classification")

shoppers_vfold <- vfold_cv(shoppers_training, v = 5, strata = Revenue)



In [ ]:
shoppers_workflow <- workflow() %>% 
    add_recipe(shoppers_recipe) %>% 
    add_model(knn_spec) %>% 
    tune_grid(resamples = shoppers_vfold, grid = 25) %>% 
    collect_metrics()
shoppers_workflow

In [ ]:
shoppers_accuracy <- shoppers_workflow %>% 
    filter(.metric == "accuracy")
shoppers_k <- shoppers_accuracy %>% 
    ggplot(aes(x = neighbors, y = mean)) +
    geom_line()+
    geom_point()
shoppers_k